In [29]:
from dotenv import load_dotenv
load_dotenv()
import os
import google.generativeai as genai
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

In [ ]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag", dimension=768, metric="cosine", spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  ),
  deletion_protection="disabled"
)

In [31]:
import json 
data = json.load(open("reviews.json"))

data["reviews"]



[{'state': 'California',
  'city': 'Los Angeles',
  'stars': 5,
  'review': 'A vibrant city with endless entertainment and fantastic weather!'},
 {'state': 'New York',
  'city': 'New York City',
  'stars': 4,
  'review': 'Great for nightlife and culture, but traffic is a nightmare.'},
 {'state': 'Texas',
  'city': 'Austin',
  'stars': 5,
  'review': 'A fun, music-filled city with amazing food and welcoming people.'},
 {'state': 'Illinois',
  'city': 'Chicago',
  'stars': 4,
  'review': 'Beautiful architecture and deep-dish pizza, but winters are harsh.'},
 {'state': 'Florida',
  'city': 'Miami',
  'stars': 3,
  'review': "Lovely beaches, but it's too hot and humid most of the year."},
 {'state': 'Nevada',
  'city': 'Las Vegas',
  'stars': 4,
  'review': 'Perfect for a quick getaway, though it can get overwhelming.'},
 {'state': 'Colorado',
  'city': 'Denver',
  'stars': 5,
  'review': 'Great for outdoor enthusiasts, with stunning views and friendly locals.'},
 {'state': 'Washington',
 

In [ ]:
#create embeddings

genai.configure(api_key=os.getenv("GEMINIAI_API_KEY"))
processed_data = []
client = genai

# Assuming 'client' is already instantiated and correctly configured
for review in data['reviews']:
    # Ensure the review text is being passed correctly
    model = "models/text-embedding-004"
    embedding = genai.embed_content(model=model,
                                content=review['review'],
                                task_type="retrieval_document",
                                )
    # response = client.generate_embeddings(
    #     input=review['review'],
    #     model="text-embedding-005",

    # )

    #embedding = response.data[0].embedding # Assuming the response contains the embeddings
    # Process or store the embeddings as needed


    #individual piece of data
    processed_data.append({
        "values": embedding,
        "id": review["state"],
        "metadata": {
            "review": review["review"],
            "city": review["city"],
            "stars" : review["stars"]
        }
    })


{'embedding': [0.012304764, 0.018752133, 0.008979598, -0.03754818, 0.018326351, -0.04676483, -0.006248248, -0.0256721, 0.020411972, 0.049672578, -0.028578928, 0.024019217, 0.0634724, 0.0021423546, 0.013359313, -0.0055283788, -0.01875836, 0.006114825, -0.12044967, -0.013200333, 0.023251988, -0.029555164, 0.055228706, -0.01968303, -0.006913252, -0.0056698527, -0.006085882, 0.022584595, -0.0034563069, 0.024124064, 0.01231061, 0.06433503, 0.0056542116, -0.034544364, -0.019815378, 0.0023503513, 0.027825987, -0.08198869, 0.045915235, -0.02187665, -0.06927587, 0.06391765, 0.0020550808, 0.0075251674, -0.0600888, -0.05789757, -0.0265676, 0.025426881, -0.02059692, 0.04785431, -0.009365851, 0.038206343, -0.025013816, 0.06525866, -0.042699542, -0.007207529, -0.050299663, 0.0076239053, 0.069754355, 0.011871056, 0.034144096, 0.009854381, -0.009472278, -0.026890958, 0.023199743, -0.048256915, -0.0007544907, 0.018302022, -0.042565193, 0.0406249, -0.009362313, 0.057616387, -0.0059054103, 0.043713097, 0

In [46]:
processed_data[0]

{'values': {'embedding': [0.012304764,
   0.018752133,
   0.008979598,
   -0.03754818,
   0.018326351,
   -0.04676483,
   -0.006248248,
   -0.0256721,
   0.020411972,
   0.049672578,
   -0.028578928,
   0.024019217,
   0.0634724,
   0.0021423546,
   0.013359313,
   -0.0055283788,
   -0.01875836,
   0.006114825,
   -0.12044967,
   -0.013200333,
   0.023251988,
   -0.029555164,
   0.055228706,
   -0.01968303,
   -0.006913252,
   -0.0056698527,
   -0.006085882,
   0.022584595,
   -0.0034563069,
   0.024124064,
   0.01231061,
   0.06433503,
   0.0056542116,
   -0.034544364,
   -0.019815378,
   0.0023503513,
   0.027825987,
   -0.08198869,
   0.045915235,
   -0.02187665,
   -0.06927587,
   0.06391765,
   0.0020550808,
   0.0075251674,
   -0.0600888,
   -0.05789757,
   -0.0265676,
   0.025426881,
   -0.02059692,
   0.04785431,
   -0.009365851,
   0.038206343,
   -0.025013816,
   0.06525866,
   -0.042699542,
   -0.007207529,
   -0.050299663,
   0.0076239053,
   0.069754355,
   0.011871056,
  

In [50]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"

)

TypeError: Column `values` is expected to be a list of floats